<a href="https://colab.research.google.com/github/shishiradk/pytorch-pratices/blob/main/03_Pytorch_Computer_Vision.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Pytorch  Computer Vision

## 0. Compuer vision libraries  in Pytorch

* `torchvision` - base domain library for Pytorch computer vision
* `torchvision.datasets`  - get  datasets and data loading functions for computer vision here
* `torchvision.models` - get pretrained computer vision models that you can leverage for your own problems
*  `torchvision.transforms` - functions for manipulating your vision data (images) to be suitable for se with an ML model
* `torch.utils.data.Dataset`  - Base dataset class for Pytorch.
* `torch.utils.data.dataloader` - Creates a Python iterable over a dataset

In [1]:
# import Pytorch
import torch
from torch import nn

#import torchvision
import torchvision

from torchvision import datasets
from torchvision import transforms
from torchvision.transforms import ToTensor

#import matplotlib for visualization
print(torch.__version__)
print(torchvision.__version__)

2.8.0+cu126
0.23.0+cu126
